In [1]:
#Importing data modules
import numpy as np
import scipy.stats as stats
import pandas as pd


In [2]:
fighters = pd.read_csv('Data Sets/Updated_fighters_cleaned.csv')

In [3]:
#Initial assesment of the data
print fighters.shape
fighters.head()

(1522, 11)


,url,fid,name,nick,birth_date,height,weight,association,class,locality,country
0,/fighter/Ken-Shamrock-4,4,Ken Shamrock,The World's Most Dangerous Man,2/11/1964,73.0,212.0,Lion's Den,Heavyweight,"San Diego, California",United States
1,/fighter/Carlos-Newton-7,7,Carlos Newton,The Ronin,8/17/1976,69.0,174.0,Warrior MMA Fight Team,Middleweight,"Newmarket, Ontario",Canada
2,/fighter/Ricco-Rodriguez-8,8,Ricco Rodriguez,Suave,8/19/1977,76.0,214.0,American Fight Club,Heavyweight,"Staten Island, New York",United States
3,/fighter/John-Marsh-9,9,John Marsh,The Bull,9/15/1970,72.0,222.0,Team Neutral Grounds,Heavyweight,"Torrance, California",United States
4,/fighter/Ricardo-Almeida-11,11,Ricardo Almeida,Big Dog,11/29/1976,71.0,170.0,Renzo Gracie Jiu-Jitsu,Welterweight,"Bordentown, New Jersey",United States


In [4]:
#I dropped the URL column as this was only included to help in the scraping process
fighters.drop(['url'], axis=1, inplace=True)

### Filling in missing data

Most of the fighers contained in the data set have complete data, but there are a handful of fighters that needed to have missing vaues manually added. I created the functions in the cell below to add in values. Most of the missing values I was able to find on http://www.ufc.com/, http://www.mixedmartialarts.com/ or http://www.tapology.com/.

In [5]:
# Creating functions to manually add in missing values for speicifc fighters
def height_changer(index, newval):
    fighters.set_value(index,'height', newval)
def weight_changer(index, newval):
    fighters.set_value(index, 'weight', newval)
def class_changer(index, newval):
    fighters.set_value(index, 'class', newval)
def locale_changer(index, newval):
    fighters.set_value(index, 'locality', newval)
def country_changer(index, newval):
    fighters.set_value(index, 'country', newval)

In [6]:
#Adding country values
country_changer(1199, 'United States')
country_changer(839, 'Brazil')
country_changer(1325, 'United States')
country_changer(1399, "United States")
locale_changer(1399, 'Iowa City')

In [7]:
#Adding in fighter locale
locale_changer(1199, 'Memphis')
locale_changer(1399, 'Iowa City')

In [8]:
#updating heights
height_changer(4, 74)
height_changer(31, 71)
height_changer(75, 71)
height_changer(83, 72)
height_changer(95, 68)
height_changer(393, 72)
height_changer(495, 65)
height_changer(502, 69)
height_changer(513, 70)
height_changer(567, 69)
height_changer(718, 73)
height_changer(858, 72)
height_changer(885, 74)
height_changer(902, 75)
height_changer(911, 69)
height_changer(926, 72)
height_changer(933, 66)
height_changer(1001, 71)
height_changer(1018, 71)
height_changer(1066, 69)
height_changer(1101, 67)
height_changer(1170, 67)
height_changer(1399, 70)

In [9]:
#updating weights
weight_changer(885, 170)
weight_changer(902, 250)
weight_changer(926, 225)
weight_changer(1001, 230)
weight_changer(1018, 155)
weight_changer(1397, 217)
weight_changer(1399, 200)

In [10]:
#adding in weight class
class_changer(35, 'Featherweight')
class_changer(66, 'Featherweight')
class_changer(358, 'Featherweight')
class_changer(902, 'Heavyweight')
class_changer(926, 'Heavyweight')
class_changer(1001, 'Heavyweight')
class_changer(1439, 'Light Heavyweight')
class_changer(1399, 'Light Heavyweight')
class_changer(1397, 'Heavyweight')
class_changer(1391, 'Heavyweight')
class_changer(1338, 'Light Heavyweight')
class_changer(1274, 'Light Heavyweight')

In [11]:
#Checking for remaining nulls
fighters.isnull().sum()

fid              0
name             0
nick           489
birth_date     203
height          48
weight          26
association    189
class           33
locality        57
country         25
dtype: int64

For the remaining height and weight nulls I just decided to use the mean.

In [12]:
#Filling nulls height and weight with the mean of height and weight
fighters['weight'].fillna(fighters['weight'].mean(), inplace=True)
fighters['height'].fillna(fighters['height'].mean(), inplace=True)

For fighters with a country and not a locale, I just made the country their locale.

In [13]:
for i in range(len(fighters)):
    if pd.isnull(fighters['locality'][i]) == True:
        fighters['locality'][i] = fighters['country'][i]
    else: 
        pass

/Users/mikesanders/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


I created the function below to fill in the NaN values in the columns that were string objects.

-For fighters with no nickname, the string 'None' was added.

-'Unassociated' was added with no formal gym, team or training facility association.

-If both the locale and country were not available online, then both strings were changed to 'Unknown
'
-Since the mean weight fell within the middleweight class, fighters who had the mean imputed for their unknown weight were also assigned the class 'Middleweight'.

In [14]:
#function to replace NaNs with a substituted string of choice.
def denaner(x, y):
    for i in range(len(fighters)):
        if pd.isnull(fighters[x][i]) == True:
            fighters[x][i] = y
        else: 
            pass

In [15]:
denaner('nick', 'None')
denaner('association', 'Unassociated')
denaner('locality', 'Unknown')
denaner('country', 'Unknown')
denaner('class', 'Middleweight')


/Users/mikesanders/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Two fighters were completely missing from the fighter data frame so I had to create the temporary one below to add them to the main figher data frame.

In [16]:
#Creating a temporary data frame with two fighters who were missing

new_fighters = pd.DataFrame([[59766, 'Mirsad Bektic', 'None', '2/16/1991', 68, 145, 'American Top Team', 'Featherweight', 'Cocunut Creek, Florida', 'United States'],
                             [35732, 'Noad Lahat', 'None', '6/8/1984', 69, 145, 'American Kickboxing Academy', 'Featherweight', 'San Jose, California', 'United States']],
                             columns=['fid', 'name', 'nick', 'birth_date', 'height', 'weight', 'association', 'class', 'locality', 'country'])

In [17]:
#Appending the two additional fighters
fighters = fighters.append(new_fighters)

To smooth the merging process and avoid any conflicts when setting out to do the modeling, I creted identical copies of the fighters data frame. The tag 'f1' would be added to the columns in the 'fighters_f1' dataframe and likewise with 'f2' and the 'fighter_f2' data frame.

In [18]:
#making two identical data frames which are copies of the initial fighters data frame to help with merging.

fighters_f1 = pd.DataFrame(fighters.values)
fighters_f2 = pd.DataFrame(fighters.values)

In [19]:
#Adding f1 to the head of all column names to help with merging
fighters_f1.columns = ['f1fid', 'f1name', 'f1nick', 'f1birth_date', 'f1height', 'f1weight',
       'f1association', 'f1class', 'f1locality', 'f1country']

In [20]:
fighters_f1.head()

,f1fid,f1name,f1nick,f1birth_date,f1height,f1weight,f1association,f1class,f1locality,f1country
0,4,Ken Shamrock,The World's Most Dangerous Man,2/11/1964,73,212,Lion's Den,Heavyweight,"San Diego, California",United States
1,7,Carlos Newton,The Ronin,8/17/1976,69,174,Warrior MMA Fight Team,Middleweight,"Newmarket, Ontario",Canada
2,8,Ricco Rodriguez,Suave,8/19/1977,76,214,American Fight Club,Heavyweight,"Staten Island, New York",United States
3,9,John Marsh,The Bull,9/15/1970,72,222,Team Neutral Grounds,Heavyweight,"Torrance, California",United States
4,11,Ricardo Almeida,Big Dog,11/29/1976,74,170,Renzo Gracie Jiu-Jitsu,Welterweight,"Bordentown, New Jersey",United States


In [21]:
# Adding f2 to all fighter names to help with merging
fighters_f2.columns = ['f2fid', 'f2name', 'f2nick', 'f2birth_date', 'f2height', 'f2weight',
       'f2association', 'f2class', 'f2locality', 'f2country']

In [22]:
fighters_f2.head()

,f2fid,f2name,f2nick,f2birth_date,f2height,f2weight,f2association,f2class,f2locality,f2country
0,4,Ken Shamrock,The World's Most Dangerous Man,2/11/1964,73,212,Lion's Den,Heavyweight,"San Diego, California",United States
1,7,Carlos Newton,The Ronin,8/17/1976,69,174,Warrior MMA Fight Team,Middleweight,"Newmarket, Ontario",Canada
2,8,Ricco Rodriguez,Suave,8/19/1977,76,214,American Fight Club,Heavyweight,"Staten Island, New York",United States
3,9,John Marsh,The Bull,9/15/1970,72,222,Team Neutral Grounds,Heavyweight,"Torrance, California",United States
4,11,Ricardo Almeida,Big Dog,11/29/1976,74,170,Renzo Gracie Jiu-Jitsu,Welterweight,"Bordentown, New Jersey",United States


### Reading in the fights data frame to do cleaning and prep for merging with the fighters data frame.

In [23]:
fights = pd.read_csv('Data Sets/ALL UFC FIGHTS 2-23-2016 SHERDOG.COM - Sheet1.csv')

In [24]:
fights.head()

,pageurl,eid,mid,event_name,event_org,event_date,event_place,f1pageurl,f2pageurl,f1name,f2name,f1result,f2result,f1fid,f2fid,method,method_d,ref,round,time
0,/events/UFC-1-The-Beginning-7,7,8,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Royce-Gracie-19,/fighter/Gerard-Gordeau-15,Royce Gracie,Gerard Gordeau,win,loss,19,15,Submission,Rear-Naked Choke,Helio Vigio,1,1:44
1,/events/UFC-1-The-Beginning-7,7,7,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Jason-DeLucia-22,/fighter/Trent-Jenkins-23,Jason DeLucia,Trent Jenkins,win,loss,22,23,Submission,Rear-Naked Choke,Joao Alberto Barreto,1,0:52
2,/events/UFC-1-The-Beginning-7,7,6,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Royce-Gracie-19,/fighter/Ken-Shamrock-4,Royce Gracie,Ken Shamrock,win,loss,19,4,Submission,Rear-Naked Choke,Helio Vigio,1,0:57
3,/events/UFC-1-The-Beginning-7,7,5,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Gerard-Gordeau-15,/fighter/Kevin-Rosier-17,Gerard Gordeau,Kevin Rosier,win,loss,15,17,TKO,Corner Stoppage,Joao Alberto Barreto,1,0:59
4,/events/UFC-1-The-Beginning-7,7,4,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Ken-Shamrock-4,/fighter/Patrick-Smith-21,Ken Shamrock,Patrick Smith,win,loss,4,21,Submission,Heel Hook,Helio Vigio,1,1:49


In [25]:
#Checking for nulls
fights.isnull().sum()

pageurl          0
eid              0
mid              0
event_name       0
event_org        0
event_date       0
event_place      0
f1pageurl        0
f2pageurl        0
f1name           0
f2name           0
f1result         0
f2result         0
f1fid            0
f2fid            0
method           0
method_d        20
ref            321
round            0
time             1
dtype: int64

In [81]:
#Checking for the location of nulls in the sub-method category
fights.ix[pd.isnull(fights['method_d'])]

,event_date,event_place,f1result,f2result,f1fid,f2fid,method,method_d,ref,round,time


In [27]:
fights['method'].value_counts()

Decision                                       1470
TKO                                             779
Submission                                      768
KO                                              414
Technical                                        68
NC                                               26
Draw                                             21
DQ                                               11
No                                                6
No Contest                                        2
No Contest - Overturned by Commission             1
No Contest - Caceres Failed Drug Test             1
No Contest - Overturned by State Commission       1
No Contest - Hoffman Failed Drug Test             1
Name: method, dtype: int64

In [29]:
def method_changer(index, newval):
    fights.set_value(index, 'method_d', newval)

In [30]:
# adding in the sub-method when the sub-method was not listed
method_changer(37, 'Knockout')
method_changer(143, 'Unanimous')
method_changer(146, 'Split')
method_changer(167, 'Split')
method_changer(168, 'Split')
method_changer(170, 'Punches')
method_changer(171, 'Punches')
method_changer(40, 'Time Limit Draw')
method_changer(60, 'Time Limit Draw')
method_changer(202, 'Time Limit Draw')
method_changer(208, 'Decision Overturned')
method_changer(237, 'Time Limit Draw')
method_changer(347, 'Split Decision')
method_changer(365, 'Time Limit Draw')
method_changer(2558, 'Failed Drug Test')
method_changer(257, 'Failed Drug Test')
method_changer(2210, 'Failed Drug Test')
method_changer(2461, 'Failed Drug Test')
method_changer(2265, 'Failed Drug Test')
method_changer(314, 'Decision Overturned')

In [31]:
for i in range(len(fights)):
    if pd.isnull(fights['ref'][i]) == True:
        fights['ref'][i] = 'Unknown'
    else: 
        pass


/Users/mikesanders/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [32]:
for i in range(len(fights)):
    if pd.isnull(fights['time'][i]) == True:
        fights['time'][i] = '10:01'
    else: 
        pass


/Users/mikesanders/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [33]:
#Dropping columns I do not need
fights.drop(['pageurl', 'eid','mid', 'event_org', 
             'f1pageurl', 'f2pageurl', 'event_name', 'f1name', 'f2name'], axis=1, inplace=True)


In [34]:
#Making sure I do not add in extra rows when merging

print fights['f1result'].value_counts()
print fights['f2result'].value_counts()

win     3510
NC        38
draw      21
Name: f1result, dtype: int64
loss    3510
NC        38
draw      21
Name: f2result, dtype: int64


In [35]:
fights.shape

(3569, 11)

In [36]:
#Merging in the first fighters table
combo_fights_temp = fights.merge(fighters_f1, how='left', on='f1fid')

In [37]:
combo_fights_temp.head()

,event_date,event_place,f1result,f2result,f1fid,f2fid,method,method_d,ref,round,time,f1name,f1nick,f1birth_date,f1height,f1weight,f1association,f1class,f1locality,f1country
0,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,19,15,Submission,Rear-Naked Choke,Helio Vigio,1,1:44,Royce Gracie,None,12/12/1966,72,176,Gracie Humaita,Middleweight,Rio de Janeiro,Brazil
1,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,22,23,Submission,Rear-Naked Choke,Joao Alberto Barreto,1,0:52,Jason DeLucia,None,7/24/1969,71,190,Unassociated,Light Heavyweight,"Bellingham, Massachusets",United States
2,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,19,4,Submission,Rear-Naked Choke,Helio Vigio,1,0:57,Royce Gracie,None,12/12/1966,72,176,Gracie Humaita,Middleweight,Rio de Janeiro,Brazil
3,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,15,17,TKO,Corner Stoppage,Joao Alberto Barreto,1,0:59,Gerard Gordeau,None,3/30/1955,77,216,Kamakura,Heavyweight,The Hague,Netherlands
4,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,4,21,Submission,Heel Hook,Helio Vigio,1,1:49,Ken Shamrock,The World's Most Dangerous Man,2/11/1964,73,212,Lion's Den,Heavyweight,"San Diego, California",United States


In [38]:
combo_fights_temp.shape

(3569, 20)

In [39]:
#Making sure I did not add any extra rows
print combo_fights_temp['f1result'].value_counts()
print combo_fights_temp['f2result'].value_counts()
#print combo_fights_temp['fightid'].value_counts()

win     3510
NC        38
draw      21
Name: f1result, dtype: int64
loss    3510
NC        38
draw      21
Name: f2result, dtype: int64


In [40]:
#Merging in the second fighters table to make sure
combo_fights_temp_2 = combo_fights_temp.merge(fighters_f2, how='left', on='f2fid')

In [41]:
combo_fights_temp_2.head()

,event_date,event_place,f1result,f2result,f1fid,f2fid,method,method_d,ref,round,...,f1country,f2name,f2nick,f2birth_date,f2height,f2weight,f2association,f2class,f2locality,f2country
0,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,19,15,Submission,Rear-Naked Choke,Helio Vigio,1,...,Brazil,Gerard Gordeau,None,3/30/1955,77,216,Kamakura,Heavyweight,The Hague,Netherlands
1,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,22,23,Submission,Rear-Naked Choke,Joao Alberto Barreto,1,...,United States,Trent Jenkins,None,NaN,74,185,Unassociated,Middleweight,"Denver, Colorado",United States
2,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,19,4,Submission,Rear-Naked Choke,Helio Vigio,1,...,Brazil,Ken Shamrock,The World's Most Dangerous Man,2/11/1964,73,212,Lion's Den,Heavyweight,"San Diego, California",United States
3,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,15,17,TKO,Corner Stoppage,Joao Alberto Barreto,1,...,Netherlands,Kevin Rosier,None,1/6/1962,76,265,Unassociated,Heavyweight,"Tampa, Florida",United States
4,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,4,21,Submission,Heel Hook,Helio Vigio,1,...,United States,Patrick Smith,None,7/16/1963,74,225,Unassociated,Heavyweight,"Denver, Colorado",United States


In [42]:
print combo_fights_temp_2['f1result'].value_counts()
print combo_fights_temp_2['f2result'].value_counts()

win     3510
NC        38
draw      21
Name: f1result, dtype: int64
loss    3510
NC        38
draw      21
Name: f2result, dtype: int64


In [43]:
combo_fights_temp_2.shape

(3569, 29)

In [44]:
merged_fights_fighters = combo_fights_temp_2

In [45]:
merged_fights_fighters['event_date'] = pd.to_datetime(merged_fights_fighters['event_date'])
merged_fights_fighters['f1birth_date'] = pd.to_datetime(merged_fights_fighters['f1birth_date'])
merged_fights_fighters['f2birth_date'] = pd.to_datetime(merged_fights_fighters['f2birth_date'])


In [46]:
merged_fights_fighters['f1age'] = pd.DatetimeIndex(merged_fights_fighters['event_date']).year - pd.DatetimeIndex(merged_fights_fighters['f1birth_date']).year
merged_fights_fighters['f2age'] = pd.DatetimeIndex(merged_fights_fighters['event_date']).year - pd.DatetimeIndex(merged_fights_fighters['f2birth_date']).year

In [47]:
merged_fights_fighters.dtypes

event_date       datetime64[ns]
event_place              object
f1result                 object
f2result                 object
f1fid                     int64
f2fid                     int64
method                   object
method_d                 object
ref                      object
round                     int64
time                     object
f1name                   object
f1nick                   object
f1birth_date     datetime64[ns]
f1height                 object
f1weight                 object
f1association            object
f1class                  object
f1locality               object
f1country                object
f2name                   object
f2nick                   object
f2birth_date     datetime64[ns]
f2height                 object
f2weight                 object
f2association            object
f2class                  object
f2locality               object
f2country                object
f1age                   float64
f2age                   float64
dtype: o

In [48]:
print merged_fights_fighters['f1age'].head()
print merged_fights_fighters['f2age'].head()

0    27.0
1    24.0
2    27.0
3    38.0
4    29.0
Name: f1age, dtype: float64
0    38.0
1     NaN
2    29.0
3    31.0
4    30.0
Name: f2age, dtype: float64


In [49]:
print merged_fights_fighters['f2age'].mean()
print merged_fights_fighters['f2age'].min()
print merged_fights_fighters['f2age'].max()
print merged_fights_fighters['f2age'].median()

29.8058049072
18.0
48.0
30.0


In [50]:
print merged_fights_fighters['f1age'].mean()
print merged_fights_fighters['f1age'].min()
print merged_fights_fighters['f1age'].max()
print merged_fights_fighters['f1age'].median()

29.1443298969
19.0
47.0
29.0


In [51]:
merged_fights_fighters['f2age'].fillna(merged_fights_fighters['f2age'].median(), inplace=True)
merged_fights_fighters['f1age'].fillna(merged_fights_fighters['f1age'].median(), inplace=True)

In [52]:
merged_fights_fighters.drop(['event_date', 'f1birth_date', 'f2birth_date'], axis=1, inplace=True)

In [53]:
merged_fights_fighters.isnull().sum()

event_place      0
f1result         0
f2result         0
f1fid            0
f2fid            0
method           0
method_d         0
ref              0
round            0
time             0
f1name           0
f1nick           0
f1height         0
f1weight         0
f1association    0
f1class          0
f1locality       0
f1country        0
f2name           0
f2nick           0
f2height         0
f2weight         0
f2association    0
f2class          0
f2locality       0
f2country        0
f1age            0
f2age            0
dtype: int64

In [54]:
new_time = []
for x in merged_fights_fighters['time']:
    if x =='36:00:00':
        fixed_time = '36:00'
    elif x == '33:00:00':
        fixed_time = '33:00'
    elif x == '30:00:00':
        fixed_time = '30:00'
    else:
        fixed_time = x
    new_time.append(fixed_time)

In [55]:
merged_fights_fighters['time'] = new_time


In [56]:
merged_fights_fighters['time'].head()

0    1:44
1    0:52
2    0:57
3    0:59
4    1:49
Name: time, dtype: object

In [57]:
import datetime

total_times = []
for x in new_time:
    times  = datetime.datetime.strptime(x, '%M:%S')
    mins = times.minute
    secs = times.second
    secs = secs/60.
    mins = mins+secs
    total_times.append(mins)

In [58]:

for x in total_times:
    print x

1.73333333333
0.866666666667
0.95
0.983333333333
1.81666666667
2.3
4.33333333333
0.433333333333
1.28333333333
1.51666666667
1.11666666667
1.11666666667
1.48333333333
3.11666666667
0.5
5.13333333333
6.68333333333
9.85
2.83333333333
4.86666666667
12.2166666667
0.966666666667
0.333333333333
1.45
4.56666666667
4.66666666667
0.766666666667
4.81666666667
1.98333333333
15.8166666667
2.21666666667
0.866666666667
5.53333333333
1.75
4.78333333333
2.73333333333
3.98333333333
4.91666666667
0.233333333333
3.01666666667
36.0
4.35
2.2
1.63333333333
2.38333333333
2.91666666667
1.38333333333
2.03333333333
0.35
17.75
2.23333333333
0.15
1.88333333333
0.95
1.13333333333
1.03333333333
0.333333333333
3.1
2.01666666667
13.2833333333
33.0
12.45
1.06666666667
3.38333333333
3.08333333333
1.68333333333
1.11666666667
0.516666666667
5.43333333333
0.8
2.23333333333
4.4
5.51666666667
0.8
0.216666666667
9.05
10.0
0.133333333333
0.833333333333
30.0
9.36666666667
0.666666666667
12.0
5.53333333333
3.23333333333
7.416666

In [59]:
merged_fights_fighters.head()

,event_place,f1result,f2result,f1fid,f2fid,method,method_d,ref,round,time,...,f2name,f2nick,f2height,f2weight,f2association,f2class,f2locality,f2country,f1age,f2age
0,"McNichols Arena, Denver, Colorado, United States",win,loss,19,15,Submission,Rear-Naked Choke,Helio Vigio,1,1:44,...,Gerard Gordeau,None,77,216,Kamakura,Heavyweight,The Hague,Netherlands,27.0,38.0
1,"McNichols Arena, Denver, Colorado, United States",win,loss,22,23,Submission,Rear-Naked Choke,Joao Alberto Barreto,1,0:52,...,Trent Jenkins,None,74,185,Unassociated,Middleweight,"Denver, Colorado",United States,24.0,30.0
2,"McNichols Arena, Denver, Colorado, United States",win,loss,19,4,Submission,Rear-Naked Choke,Helio Vigio,1,0:57,...,Ken Shamrock,The World's Most Dangerous Man,73,212,Lion's Den,Heavyweight,"San Diego, California",United States,27.0,29.0
3,"McNichols Arena, Denver, Colorado, United States",win,loss,15,17,TKO,Corner Stoppage,Joao Alberto Barreto,1,0:59,...,Kevin Rosier,None,76,265,Unassociated,Heavyweight,"Tampa, Florida",United States,38.0,31.0
4,"McNichols Arena, Denver, Colorado, United States",win,loss,4,21,Submission,Heel Hook,Helio Vigio,1,1:49,...,Patrick Smith,None,74,225,Unassociated,Heavyweight,"Denver, Colorado",United States,29.0,30.0


In [60]:
merged_fights_fighters['time'] = total_times

In [61]:
print merged_fights_fighters['round'].value_counts()

3    1646
1    1226
2     578
5      98
4      21
Name: round, dtype: int64


In [62]:
temp_round = []
for value in merged_fights_fighters['round']:
    x = (value - 1)* 5
    temp_round.append(x)
merged_fights_fighters['temp'] = temp_round

In [63]:
merged_fights_fighters['temp'].value_counts()

10    1646
0     1226
5      578
20      98
15      21
Name: temp, dtype: int64

In [64]:
merged_fights_fighters.tail()

,event_place,f1result,f2result,f1fid,f2fid,method,method_d,ref,round,time,...,f2nick,f2height,f2weight,f2association,f2class,f2locality,f2country,f1age,f2age,temp
3564,"Consol Energy Center, Pittsburgh, Pennsylvania...",win,loss,29470,14971,Decision,Unanimous,Chip Snider,3,5.000000,...,Leleco,72,185,MMA Masters,Middleweight,"Salvador, Bahia",Brazil,28.0,34.0,10
3565,"Consol Energy Center, Pittsburgh, Pennsylvania...",win,loss,25412,81634,Decision,Unanimous,Keith Peterson,3,5.000000,...,Spyder,69,170,Renzo Gracie Philly,Welterweight,"Marlton, New Jersey",United States,38.0,27.0,10
3566,"Consol Energy Center, Pittsburgh, Pennsylvania...",win,loss,75021,61266,Decision,Split,James Chappell,3,5.000000,...,The Bruiser,66,135,Elite Team,Bantamweight,"Visalia, California",United States,29.0,39.0,10
3567,"Consol Energy Center, Pittsburgh, Pennsylvania...",win,loss,66725,117453,TKO,Elbows and Punches,Bill Bookwalter,3,4.916667,...,None,68,145,Nor-Cal Fighting Alliance,Featherweight,"Santa Rosa, California",United States,33.0,30.0,10
3568,"Consol Energy Center, Pittsburgh, Pennsylvania...",win,loss,26808,58947,Decision,Unanimous,Chip Snider,3,5.000000,...,Freight Train,77,253,Jackson-Wink MMA,Heavyweight,"Seattle, Washington",United States,35.0,36.0,10


In [65]:
merged_fights_fighters['match_time'] = merged_fights_fighters['time'] + merged_fights_fighters['temp']


In [66]:
merged_fights_fighters['match_time'].tail()

3564    15.000000
3565    15.000000
3566    15.000000
3567    14.916667
3568    15.000000
Name: match_time, dtype: float64

In [67]:
merged_fights_fighters.drop(['time', 'round', 'temp'], axis = 1, inplace = True)

In [68]:
merged_fights_fighters.method.value_counts()

Decision                                       1470
TKO                                             779
Submission                                      768
KO                                              414
Technical                                        68
NC                                               26
Draw                                             21
DQ                                               11
No                                                6
No Contest                                        2
No Contest - Overturned by Commission             1
No Contest - Caceres Failed Drug Test             1
No Contest - Overturned by State Commission       1
No Contest - Hoffman Failed Drug Test             1
Name: method, dtype: int64

In [69]:
#Making a quick fix for the TKOs erroeously labeled Technical
fixed_method = []
for outcome in merged_fights_fighters['method']:
    if outcome == 'Technical':
        x = 'TKO'
    else:
        x = outcome
    fixed_method.append(x)

In [70]:
merged_fights_fighters['method'] = fixed_method

Based on a trend in recent years to protect fighters more, the line between TKOs and KOs is somewhat blurred so for the interest of increasing prediction power these are being combined into one outcome.

In [75]:
new_method = []
for method in merged_fights_fighters['method']:
    if method == 'TKO':
        method = 'KO/TKO'
    elif method == 'KO':
        method = 'KO/TKO'
    else:
        method = method
    new_method.append(method)

In [76]:
merged_fights_fighters['method'] = new_method

In [77]:
merged_fights_fighters['method'].value_counts()

Decision                                       1470
KO/TKO                                         1261
Submission                                      768
NC                                               26
Draw                                             21
DQ                                               11
No                                                6
No Contest                                        2
No Contest - Overturned by Commission             1
No Contest - Caceres Failed Drug Test             1
No Contest - Overturned by State Commission       1
No Contest - Hoffman Failed Drug Test             1
Name: method, dtype: int64

In [78]:
print merged_fights_fighters.isnull().sum()
print merged_fights_fighters.columns

event_place      0
f1result         0
f2result         0
f1fid            0
f2fid            0
method           0
method_d         0
ref              0
f1name           0
f1nick           0
f1height         0
f1weight         0
f1association    0
f1class          0
f1locality       0
f1country        0
f2name           0
f2nick           0
f2height         0
f2weight         0
f2association    0
f2class          0
f2locality       0
f2country        0
f1age            0
f2age            0
match_time       0
dtype: int64
Index([u'event_place', u'f1result', u'f2result', u'f1fid', u'f2fid', u'method',
       u'method_d', u'ref', u'f1name', u'f1nick', u'f1height', u'f1weight',
       u'f1association', u'f1class', u'f1locality', u'f1country', u'f2name',
       u'f2nick', u'f2height', u'f2weight', u'f2association', u'f2class',
       u'f2locality', u'f2country', u'f1age', u'f2age', u'match_time'],
      dtype='object')


In [79]:
merged_fights_fighters.to_csv('Data Sets/Merged_Fights_Fighters.csv')